# Imports

In [1]:
import os
import json

import mlflow
from mlflow.tracking import MlflowClient
import torch
import torch.onnx
import onnx

from config import Config

# Query of experiments

In [2]:
# Initialize MLflow client
client = MlflowClient()
MODEL_NAME = Config.MODEL_NAME

2025/12/15 09:54:16 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/15 09:54:16 INFO mlflow.store.db.utils: Updating database tables
2025/12/15 09:54:16 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/15 09:54:16 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/15 09:54:17 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/15 09:54:17 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [3]:
# Get all versions of the registered model
model_versions = client.search_model_versions(f"name='{MODEL_NAME}'")

print(f"Found {len(model_versions)} registered model versions")
for version in model_versions:
    print(f"Version {version.version} - Run ID: {version.run_id}")

2025/12/15 09:54:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/15 09:54:17 INFO mlflow.store.db.utils: Updating database tables
2025/12/15 09:54:17 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/15 09:54:17 INFO alembic.runtime.migration: Will assume non-transactional DDL.


Found 6 registered model versions
Version 6 - Run ID: b679c3a119f04be4bf3d4ba4493dd3db
Version 5 - Run ID: 5cc5462ecdb84e5db9b0bae6f05a77a1
Version 4 - Run ID: 60f2b233c0704edc8d65b882ba6f356a
Version 3 - Run ID: 1d27b375f1aa4216889c338c375ef351
Version 2 - Run ID: d8f7941bcefb410b9f8b115ba7a5dfa0
Version 1 - Run ID: b6789d4cd1174d959af79611ae26f488


In [4]:
best_version = None
best_f1_score = -1

for version in model_versions:
    run_id = version.run_id
    run = client.get_run(run_id)
    metrics = run.data.metrics
    
    # Get metrics
    test_f1_score = metrics.get('test_f1_score', -1)
    
    print((
        f"Version {version.version} - "
        f"Run ID: {run_id} - "
        f"Test Acc: {test_f1_score:.4f}"
    ))
    
    if test_f1_score > best_f1_score:
        best_f1_score = test_f1_score
        best_version = version

print(f"\nBest model: Version {best_version.version}")
print(f"Run ID: {best_version.run_id}")
print(f"Test Accuracy: {best_f1_score:.4f}")

Version 6 - Run ID: b679c3a119f04be4bf3d4ba4493dd3db - Test Acc: 0.9891
Version 5 - Run ID: 5cc5462ecdb84e5db9b0bae6f05a77a1 - Test Acc: 0.9875
Version 4 - Run ID: 60f2b233c0704edc8d65b882ba6f356a - Test Acc: 0.9891
Version 3 - Run ID: 1d27b375f1aa4216889c338c375ef351 - Test Acc: 0.9876
Version 2 - Run ID: d8f7941bcefb410b9f8b115ba7a5dfa0 - Test Acc: 0.9876
Version 1 - Run ID: b6789d4cd1174d959af79611ae26f488 - Test Acc: 0.9891

Best model: Version 6
Run ID: b679c3a119f04be4bf3d4ba4493dd3db
Test Accuracy: 0.9891


# Serialization

In [5]:
# Load the model
model_uri = f"runs:/{best_version.run_id}/model"
model = mlflow.pytorch.load_model(model_uri)

# Move to CPU and set to evaluation mode
model = model.to('cpu')
model.eval()

print("Model loaded successfully and moved to CPU")

Model loaded successfully and moved to CPU


In [7]:
# dummy input for ONNX export
dummy_input = torch.randn(1, 3, 224, 224)

# Export to ONNX
onnx_model_path = f"{Config.MODEL_NAME}_{best_version.run_id[0:6]}.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    export_params=True,
    opset_version=18,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

# Convert external data to embedded format
print("Converting to embedded format...")
onnx_model = onnx.load(onnx_model_path, load_external_data=True)
onnx.save(onnx_model, onnx_model_path)

# Remove the external data file if it exists
external_data_path = onnx_model_path + ".data"
if os.path.exists(external_data_path):
    os.remove(external_data_path)
    print(f"Removed {external_data_path}")

print(f"Model serialized to {onnx_model_path} (embedded weights)")

/tmp/ipykernel_233397/212904453.py:6: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `MobileNetV2([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `MobileNetV2([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 105 of general pattern rewrite rules.
Converting to embedded format...
Removed cats_and_dogs_mobilenet_v2_b679c3.onnx.data
Model serialized to cats_and_dogs_mobilenet_v2_b679c3.onnx (embedded weights)


# Class names

In [8]:
artifact_path = "labels.json"
local_path = client.download_artifacts(best_version.run_id, artifact_path)

print(f"Class labels downloaded to: {local_path}")

# Load the class labels
with open(local_path, 'r') as f:
    class_labels = json.load(f)

# Save to a new file for production use
output_labels_path = f"{onnx_model_path.replace('.onnx', '')}_labels.json"
with open(output_labels_path, 'w') as f:
    json.dump(class_labels, f, indent=2)

print(f"Class labels saved to {output_labels_path}")

best_params = client.get_run(best_version.run_id).data.params
best_params_path = f"{onnx_model_path.replace('.onnx', '')}_params.json"
with open(best_params_path, 'w') as f:
    json.dump(best_params, f, indent=2)

print(f"Params saved to {output_labels_path}")

Class labels downloaded to: /tmp/tmpnv4uefhq/labels.json
Class labels saved to cats_and_dogs_mobilenet_v2_b679c3_labels.json
Params saved to cats_and_dogs_mobilenet_v2_b679c3_labels.json
